In [2]:
import pandas as pd 
import plotly.express as px
dfo = pd.read_csv("..\\Data\\SmA-Four-Tank-Batch-Process_V2.csv",sep=';')
dfo['timestamp']=pd.to_datetime(dfo['timestamp'])
#df_day=dfo.copy()
#df_day['h']=df_day['timestamp'].dt.hour

In [7]:
#px.scatter(dfo,x='timestamp',y='DeviationID ValueY',color='DeviationID ValueY',marginal_x="rug")

In [3]:
df=dfo.copy()
df=df.drop(columns=["timestamp","CuStepNo ValueY"])
c=list(df.columns)
c[0]="Dev"
df.columns=c
df=df[df['Dev']!=0]
df['Dev']=df['Dev']!=1
df['Dev']=df['Dev'].astype('int')
df_clean=df.copy()

In [13]:

df_clean=df_clean.iloc[:270000,]

In [14]:
df_clean.to_parquet('..\\Data\\data_01_clean.parquet')

In [3]:
pd.read_parquet('https://github.com/kiya1013/APuPDaten/blob/main/Data/data_01_clean.parquet')

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, recall_score, precision_score, accuracy_score

In [10]:
X_train,X_test,y_train,y_test=train_test_split(df_clean.drop(columns=['Dev']),df_clean['Dev'],shuffle=True,random_state=42)

In [11]:
model=RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)

RandomForestClassifier()

In [12]:
y_pred=model.predict(X_train)
print(f'R2:{r2_score(y_train,y_pred)}\nAccuracy: {accuracy_score(y_train,y_pred)}\nRecall: {recall_score(y_train,y_pred)}\nPrecision: {precision_score(y_train,y_pred)}')

R2:1.0
Accuracy: 1.0
Recall: 1.0
Precision: 1.0


In [13]:
y_pred=model.predict(X_train)
tn, fp, fn, tp = confusion_matrix(y_train,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 27857
FP: 0
FN: 0
TP: 180989


In [14]:
y_pred=model.predict(X_test)
print(f'R2:{r2_score(y_test,y_pred)}\nAccuracy: {accuracy_score(y_test,y_pred)}\nRecall: {recall_score(y_test,y_pred)}\nPrecision: {precision_score(y_test,y_pred)}')


R2:0.866196398009059
Accuracy: 0.9844863249827626
Recall: 0.9930344793273297
Precision: 0.9890975766886367


In [15]:
y_pred=model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 8659
FP: 660
FN: 420
TP: 59877


In [19]:
l=[]
for max_depth in range(10,100,10):
    #for max_f in range(1,40,5):
    print(max_depth)
    model=RandomForestClassifier(n_estimators=100,max_depth=max_depth,)#max_features=max_f)
    model.fit(X_train,y_train)
    
    y_pred=model.predict(X_train)
    y_pred_test=model.predict(X_test)
    
    l.append({
    'R2':r2_score(y_train,y_pred),
    'Accuracy':accuracy_score(y_train,y_pred),
    'Recall':recall_score(y_train,y_pred),
    'Precision':precision_score(y_train,y_pred),
    'Test_R2':r2_score(y_test,y_pred_test),
    'Test_Accuracy':accuracy_score(y_test,y_pred_test),
    'Test_Recall':recall_score(y_test,y_pred_test),
    'Test_Precision':precision_score(y_test,y_pred_test),
    })

10
20
30
40
50
60
70
80
90


In [20]:
import plotly.express as px
df=pd.DataFrame(l)
px.line(df)